<a href="https://colab.research.google.com/github/rawar/tensorflow-notebooks/blob/master/stock_price_prediction_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predict stock prices  with Long short-term memory (LSTM)

This simple example will show you the 

## Install requirements
We install Tensorflow 2.0 with GPU support first

In [1]:
!pip install tensorflow-gpu==2.0.0-alpha0

In [4]:
!pip install pandas-datareader

## Introduction

LSTMs are very powerful in sequence prediction problems. They can store past information.

## Loading the dataset
I use pandas-datareader to get the historical stock prices from Yahoo! finance  

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas_datareader import data

In [0]:
tickers = 'AAPL'

start_date = '1980-12-01'
end_date = '2019-03-29'

stock_data = data.get_data_yahoo(tickers, start_date, end_date)

In [67]:
stock_data.head(10)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
1980-12-12,0.515625,0.513393,0.513393,0.513393,117258400.0,0.023007
1980-12-15,0.488839,0.486607,0.488839,0.486607,43971200.0,0.021807
1980-12-16,0.453125,0.450893,0.453125,0.450893,26432000.0,0.020206
1980-12-17,0.464286,0.462054,0.462054,0.462054,21610400.0,0.020706
1980-12-18,0.477679,0.475446,0.475446,0.475446,18362400.0,0.021307
1980-12-19,0.506696,0.504464,0.504464,0.504464,12157600.0,0.022607
1980-12-22,0.531250,0.529018,0.529018,0.529018,9340800.0,0.023707
1980-12-23,0.553571,0.551339,0.551339,0.551339,11737600.0,0.024708
1980-12-24,0.582589,0.580357,0.580357,0.580357,12000800.0,0.026008


I'm only interested in *close* prices

In [0]:
close_prices = stock_data.iloc[:, 1:2].values

Of course, some of the weekdays might be public holidays in which case no price will be available. For this reason, we will fill the missing prices with the latest available prices

In [0]:
#all_weekdays = pd.date_range(start=start_date, end=end_date, freq='B')
#close_prices = close_prices.reindex(all_weekdays)
#close_prices = close_prices.fillna(method='ffill')

In [0]:
#print(all_weekdays)

The dataset is now complete and free of missing values. Let's have a look to the data frame summary:

## Feature scaling

In [0]:
training_set = stock_data.loc[:, 'Close']

In [0]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(close_prices)

LSTMs expect the data in a specific format, usually a 3D array (tensor). I start by creating data in 60 timesteps and converting it into an array using NumPy. Next, I convert the data into a 3D dimension array with X_train samples, 60 timestamps, and one feature at each step.

In [0]:
X_train = []
y_train = []
for i in range(60, 2035):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

## Create the LSTM network
Let's create a sequenced LSTM network with 50 units. Also the net includes some dropout layers with 0.2 which means that 20% of the layers will be dropped.

In [0]:
import tensorflow as tf

In [85]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units = 50, return_sequences = True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units = 50, return_sequences = True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units = 50),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units = 1)
])

W0329 15:34:27.473214 140264946407296 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f91823cc438>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0329 15:34:27.498182 140264946407296 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f91ce822128>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0329 15:34:27.506912 140264946407296 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f91ce84bb00>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0329 15:34:27.511302 140264946407296 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f91802d1b38>: Note that this layer is not optimized for performance. Please use tf.keras.laye

The model will be compiled and optimize by the adam optimizer and set the loss function as mean_squarred_error

In [86]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
1975/1975 [==============================] - 5s 3ms/sample - loss: 1.5193e-05
Epoch 2/100
1975/1975 [==============================] - 4s 2ms/sample - loss: 3.8667e-06
Epoch 3/100
1975/1975 [==============================] - 4s 2ms/sample - loss: 2.8390e-06
Epoch 4/100
1975/1975 [==============================] - 4s 2ms/sample - loss: 9.2604e-07
Epoch 5/100
1975/1975 [==============================] - 4s 2ms/sample - loss: 5.7586e-07
Epoch 6/100
1975/1975 [==============================] - 4s 2ms/sample - loss: 6.9929e-07
Epoch 7/100
1975/1975 [==============================] - 4s 2ms/sample - loss: 4.8375e-07
Epoch 8/100
1975/1975 [==============================] - 4s 2ms/sample - loss: 3.9812e-07
Epoch 9/100
1975/1975 [==============================] - 4s 2ms/sample - loss: 5.6590e-07
Epoch 10/100
1975/1975 [==============================] - 4s 2ms/sample - loss: 6.7315e-07
Epoch 11/100
1975/1975 [==============================] - 4s 2ms/sample - loss: 4.7882e-07
Epoch 12